<a href="https://colab.research.google.com/github/abdeladime2003/deep_learning/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dataset from Our Google Drive and unzip it
"""from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/archive (8).zip" -d "/content/dataset"""

'from google.colab import drive\ndrive.mount(\'/content/drive\')\n!unzip "/content/drive/MyDrive/archive (8).zip" -d "/content/dataset'

In [ ]:
import tensorflow as tf
training_path = "/content/dataset/real_vs_fake/real-vs-fake/train"
validation_path = "/content/dataset/real_vs_fake/real-vs-fake/valid"
img_size = (224, 224)  
batch_size = 32 
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    training_path,
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    shuffle=True,
    batch_size=batch_size
)
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_path,
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    shuffle=True,
    batch_size=batch_size
)
def preprocess_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label
train_dataset = train_dataset.map(preprocess_image)
validation_dataset = validation_dataset.map(preprocess_image)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)


In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/real_vs_fake/real-vs-fake/test',  
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    batch_size=batch_size,
)
test_dataset = test_dataset.map(preprocess_image)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers #type:ignore
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler #type:ignore
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False
)
base_model.trainable = True
fine_tune_at = 20  
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid") 
])


model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,770,433 (94.49 MB)

 Trainable params: 24,614,593 (93.90 MB)

 Non-trainable params: 155,840 (608.75 KB)

In [ ]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10, 
    callbacks=[early_stopping]
)



In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
import numpy as np
image_path = "/content/dataset/real_vs_fake/real-vs-fake/test/real/00277.jpg"
img = tf.keras.preprocessing.image.load_img(image_path, target_size=img_size)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) 
img_array = preprocess_image(img_array, 0)[0]
prediction = model.predict(img_array)
if prediction > 0.5:
    print("Real image")
else:
    print("Fake image")


In [ ]:
model.save('mon_modele.keras')


In [ ]:
import matplotlib.pyplot as plt #type:ignore
import numpy as np
import tensorflow as tf

def preprocess_image(image, label=None): 
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label if label is not None else image 

img_path = "/content/img/seed1000028.png"
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  
img_array = preprocess_image(img_array) 
prediction = model.predict(img_array)
print(prediction)
if prediction > 0.5:
    print("Real image")
else:
    print("Fake image")